In [1]:
!pip3 install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 782.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.6/500.6 KB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 509.4 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 2.0 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import itertools
import time

In [3]:
POSSIBLE_VALUES = set(range(1, 10))
POSSIBLE_VALUES

{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [4]:
BLOQUES = [
    set(itertools.product([0, 1, 2], ['A', 'B', 'C'])),
    set(itertools.product([0, 1, 2], ['D', 'E', 'F'])),
    set(itertools.product([0, 1, 2], ['G', 'H', 'I'])),

    set(itertools.product([3, 4, 5], ['A', 'B', 'C'])),
    set(itertools.product([3, 4, 5], ['D', 'E', 'F'])),
    set(itertools.product([3, 4, 5], ['G', 'H', 'I'])),

    set(itertools.product([6, 7, 8], ['A', 'B', 'C'])),
    set(itertools.product([6, 7, 8], ['D', 'E', 'F'])),
    set(itertools.product([6, 7, 8], ['G', 'H', 'I'])),
]

In [20]:
sudoku_df = pd.read_csv('sudoku_input_hard.csv', header=None)
sudoku_df.columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
sudoku_df

,A,B,C,D,E,F,G,H,I
0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0
1,NaN,NaN,9.0,5.0,NaN,1.0,NaN,NaN,7.0
2,NaN,NaN,NaN,NaN,9.0,NaN,NaN,3.0,NaN
3,NaN,6.0,8.0,NaN,2.0,4.0,NaN,7.0,NaN
4,NaN,NaN,3.0,NaN,NaN,NaN,9.0,NaN,NaN
5,NaN,5.0,NaN,8.0,1.0,NaN,6.0,2.0,NaN
6,NaN,1.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN
7,7.0,NaN,NaN,6.0,NaN,9.0,1.0,NaN,NaN
8,2.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN


In [21]:
sudoku_df.fillna('', inplace=True)
sudoku_df

,A,B,C,D,E,F,G,H,I
0,,7,,,,,,,9
1,,,9,5,,1,,,7
2,,,,,9,,,3,
3,,6,8,,2,4,,7,
4,,,3,,,,9,,
5,,5,,8,1,,6,2,
6,,1,,,3,,,,
7,7,,,6,,9,1,,
8,2,,,,,,,5,


In [8]:
def get_indexes(row_index, column_index):
    index_blocks = [b for b in BLOQUES if (row_index, column_index) in b][0]
    column_indexes = {(row_index, c_index) for c_index in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']}
    row_indexes = {(r_index, column_index) for r_index in range(9)}

    return index_blocks.union(column_indexes).union(row_indexes)

In [9]:
def get_values_in_indexes(indexes, df):
    values_set = set()
    for r_i, c_i in indexes:
        value_cell = df.iloc[r_i][c_i]
        if not (isinstance(value_cell, str) or isinstance(value_cell, set)):
            values_set = values_set.union({value_cell})
    return values_set

In [10]:
def get_df_sudoku(sudoku_df):
    _sudoku_df = sudoku_df.copy()
    for row_index, row in _sudoku_df.iterrows():
        for c_index in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
            cell_value_is_empty = _sudoku_df.iloc[row_index][c_index] == ''
            if cell_value_is_empty:
                used_values = get_values_in_indexes(get_indexes(row_index, c_index), _sudoku_df)
                cell_possible_values = POSSIBLE_VALUES - used_values
                if len(cell_possible_values) == 0:
                    return None
                if len(cell_possible_values) == 1:
                    _sudoku_df.iloc[row_index][c_index] = cell_possible_values.pop()
    return _sudoku_df

In [11]:
def get_full_df_sudoku(sudoku_df):
    _sudoku_df = sudoku_df.copy()

    last = None
    new = _sudoku_df
    while new is not None and not new.equals(last):
        last = new
        new = get_df_sudoku(last)

    if new is None:
        return None
    
    if not new.eq('').any().any():
        return new

    min_set_cardinality = 8000
    min_cell_row = None
    min_cell_col = None
    possible_values = set()
    for row_index, row in new.iterrows():
        for c_index in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
            cell_value_is_empty = _sudoku_df.iloc[row_index][c_index] == ''
            if cell_value_is_empty:
                used_values = get_values_in_indexes(get_indexes(row_index, c_index), _sudoku_df)
                cell_possible_values = POSSIBLE_VALUES - used_values
                if len(cell_possible_values) < min_set_cardinality:
                    min_set_cardinality = len(cell_possible_values)
                    possible_values = cell_possible_values.copy()
                    min_cell_row = row_index
                    min_cell_col = c_index

    for val in possible_values:
        last = _sudoku_df.copy()
        last.iloc[min_cell_row][min_cell_col] = val
        new = get_full_df_sudoku(last)
        if new is None:
            continue
        if not new.eq('').any().any():
            return new
    return None

In [22]:
startedTime = time.time()
solved = get_full_df_sudoku(sudoku_df)
finishedTime = time.time()
if solved is None:
    print('not solved')
else:
    print('solved with time: {} s'.format(finishedTime - startedTime))
    print(solved)

solved with time: 5.22077202796936 s
   A  B  C  D  E  F  G  H  I
0  5  7  1  3  6  2  8  4  9
1  3  4  9  5  8  1  2  6  7
2  6  8  2  4  9  7  5  3  1
3  1  6  8  9  2  4  3  7  5
4  4  2  3  7  5  6  9  1  8
5  9  5  7  8  1  3  6  2  4
6  8  1  4  2  3  5  7  9  6
7  7  3  5  6  4  9  1  8  2
8  2  9  6  1  7  8  4  5  3
